## Grab Sample of Data

In [1]:
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow
import random

OSM_FILE = "map.osm"  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

In [ ]:
k_range = [5,7] # Parameter: take every k-th top level element, where k is a random number between the two numbers given

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n'.encode())
    output.write('<osm>\n  '.encode())

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % random.randint(k_range[0], k_range[1]) == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>'.encode())

## Audit Data

In [3]:
import audit

audit.audit(OSM_FILE, False)

## Clean Data

In [4]:
import clean

clean.process_map(OSM_FILE, validate=True)

## Import into SQL Database

In [5]:
import csv, sqlite3, pprint
from io import TextIOWrapper

con = sqlite3.connect("osm.db")
con.row_factory = sqlite3.Row
cur = con.cursor()
cur.execute("DROP TABLE IF EXISTS nodes")
cur.execute("DROP TABLE IF EXISTS nodes_tags")
cur.execute("DROP TABLE IF EXISTS ways")
cur.execute("DROP TABLE IF EXISTS ways_tags")
cur.execute("DROP TABLE IF EXISTS ways_nodes")
cur.executescript(open("schema.sql", "r").read())

import csv_to_sql

csv_to_sql.import_csv_files(['ways_tags', 'nodes', 'nodes_tags', 'ways', 'ways_nodes'], cur, True)

Sample of records from ways_tags
[{'id': 11303991, 'key': 'natural', 'type': 'regular', 'value': 'coastline'},
 {'id': 11303991, 'key': 'place', 'type': 'regular', 'value': 'islet'},
 {'id': 11303991, 'key': 'source', 'type': 'regular', 'value': 'Bing'},
 {'id': 11304451, 'key': 'natural', 'type': 'regular', 'value': 'coastline'},
 {'id': 11304451, 'key': 'source', 'type': 'regular', 'value': 'Yahoo'}]


Sample of records from nodes
[{'changeset': 12737735,
  'id': 100428663,
  'lat': -3.7544864,
  'lon': -38.4874113,
  'timestamp': '2012-08-15T14:26:53Z',
  'uid': 804310,
  'user': 'Valentino Pinna',
  'version': 2},
 {'changeset': 46201793,
  'id': 100429090,
  'lat': -3.9356392,
  'lon': -38.3037261,
  'timestamp': '2017-02-18T20:48:54Z',
  'uid': 612405,
  'user': 'Narcélio de Sá',
  'version': 3},
 {'changeset': 10818383,
  'id': 100429099,
  'lat': -3.695213,
  'lon': -38.6172759,
  'timestamp': '2012-02-28T12:53:29Z',
  'uid': 145275,
  'user': 'Paper_',
  'version': 2},
 {'chan

## Explore Database

In [6]:
# method for simplifying SQLite queries to database
# runs query, then outputs each record as a dict object, by default
def sql_query(syntax, output=True):
    cur.execute(syntax)
    if output == True:
        pprint.pprint([dict(i) for i in cur.fetchall()])

# number of unique users
sql_query("""
    SELECT COUNT(*) as users_count
    FROM (
        SELECT uid
        FROM nodes
        UNION
        SELECT uid
        FROM ways
    )
""")

[{'users_count': 457}]


In [7]:
# number of nodes
sql_query("""
    SELECT COUNT(*) as nodes_count
    FROM nodes
""")

[{'nodes_count': 210897}]


In [8]:
# number of ways
sql_query("""
    SELECT COUNT(*) as ways_count
    FROM ways
""")

[{'ways_count': 41542}]


In [9]:
# top ten nodes tags
sql_query("""
    SELECT COUNT(*) as count, key as node_key
    FROM nodes_tags
    GROUP BY key
    ORDER BY count DESC
    LIMIT 10
""")

[{'count': 10108, 'node_key': 'natural'},
 {'count': 3453, 'node_key': 'name'},
 {'count': 2904, 'node_key': 'source'},
 {'count': 2569, 'node_key': 'power'},
 {'count': 2457, 'node_key': 'street'},
 {'count': 2350, 'node_key': 'housenumber'},
 {'count': 2333, 'node_key': 'city'},
 {'count': 2269, 'node_key': 'subur'},
 {'count': 2141, 'node_key': 'highway'},
 {'count': 1380, 'node_key': 'amenity'}]


In [10]:
# top ten ways tags
sql_query("""
    SELECT COUNT(*) as count, key
    FROM ways_tags
    GROUP BY key
    ORDER BY count DESC
    LIMIT 10
""")

[{'count': 27183, 'key': 'highway'},
 {'count': 18062, 'key': 'name'},
 {'count': 7975, 'key': 'building'},
 {'count': 5147, 'key': 'oneway'},
 {'count': 2409, 'key': 'levels'},
 {'count': 1611, 'key': 'landuse'},
 {'count': 1560, 'key': 'city'},
 {'count': 1421, 'key': 'street'},
 {'count': 1386, 'key': 'material'},
 {'count': 1320, 'key': 'subur'}]


In [11]:
# top ten users by number of posts
sql_query("""
    SELECT user, uid, COUNT(*) as num
    FROM (SELECT user, uid FROM nodes UNION ALL SELECT user, uid FROM ways)
    GROUP BY uid
    ORDER BY num DESC
    LIMIT 10
""")

[{'num': 71931, 'uid': 612405, 'user': 'Narcélio de Sá'},
 {'num': 29700, 'uid': 1657554, 'user': 'anderson stella'},
 {'num': 13650, 'uid': 2507488, 'user': 'victorquaresma'},
 {'num': 9945, 'uid': 1717892, 'user': 'Daniel de Góes Nepomuceno'},
 {'num': 9012, 'uid': 510836, 'user': 'Rub21'},
 {'num': 8819, 'uid': 1856060, 'user': 'patodiez'},
 {'num': 8775, 'uid': 4393426, 'user': 'Jakson Aquino'},
 {'num': 6835,
  'uid': 1780702,
  'user': 'Victor Hugo Pereira Soares de Joinville Moura'},
 {'num': 5595, 'uid': 149876, 'user': 'dbusse'},
 {'num': 4982, 'uid': 2889318, 'user': 'elias lopes'}]


In [31]:
# top ten neighborhoods in terms of number of nodes and ways tags
sql_query("""
    SELECT COUNT(*) as count, value as neighborhood
    FROM (SELECT * FROM nodes_tags WHERE key = 'subur' UNION ALL SELECT * FROM ways_tags WHERE key = 'subur')
    GROUP BY neighborhood
    ORDER BY count DESC
""")

[{'count': 719, 'neighborhood': 'Parque Albano'},
 {'count': 451, 'neighborhood': 'Montese'},
 {'count': 218, 'neighborhood': 'Prefeito José Walter'},
 {'count': 171, 'neighborhood': 'Damas'},
 {'count': 169, 'neighborhood': 'Conjunto Ceará'},
 {'count': 149, 'neighborhood': 'Parangaba'},
 {'count': 134, 'neighborhood': 'Vila União'},
 {'count': 128, 'neighborhood': 'Mondubim'},
 {'count': 108, 'neighborhood': 'Rodolfo Teófilo'},
 {'count': 89, 'neighborhood': 'Jardim América'},
 {'count': 84, 'neighborhood': 'Parque Dois Irmãos'},
 {'count': 82, 'neighborhood': 'Maraponga'},
 {'count': 67, 'neighborhood': 'Serrinha'},
 {'count': 66, 'neighborhood': 'Parquelândia'},
 {'count': 59, 'neighborhood': 'Aldeota'},
 {'count': 56, 'neighborhood': 'Alphaville Ceará III'},
 {'count': 53, 'neighborhood': 'Planalto Ayrton Senna'},
 {'count': 52, 'neighborhood': 'Meireles'},
 {'count': 50, 'neighborhood': 'Centro'},
 {'count': 46, 'neighborhood': 'Dionísio Torres'},
 {'count': 43, 'neighborhood': '

In [13]:
# number of religion nodes identified in the database
    
sql_query("""
    SELECT COUNT(*) as religion_count
    FROM nodes_tags
    WHERE key = 'religion'
""")

[{'religion_count': 38}]


In [23]:
sql_query("""
    SELECT key
    FROM ways_tags
    GROUP BY key
    ORDER BY key
""")

[{'key': '2'},
 {'key': '2014worldcup'},
 {'key': 'AConst'},
 {'key': 'AProjetada'},
 {'key': 'Densidade'},
 {'key': 'FID'},
 {'key': 'FIXME'},
 {'key': 'Gabarito'},
 {'key': 'L1'},
 {'key': 'L2'},
 {'key': 'L3'},
 {'key': 'L4'},
 {'key': 'Profundidade'},
 {'key': 'Rotulo'},
 {'key': 'Testada'},
 {'key': 'access'},
 {'key': 'admin_level'},
 {'key': 'advertising'},
 {'key': 'aeroway'},
 {'key': 'alt_name'},
 {'key': 'amenity'},
 {'key': 'amenity_1'},
 {'key': 'anchorage:category'},
 {'key': 'angle'},
 {'key': 'area'},
 {'key': 'atm'},
 {'key': 'backward'},
 {'key': 'barrier'},
 {'key': 'bench'},
 {'key': 'bicycle'},
 {'key': 'bin'},
 {'key': 'boat'},
 {'key': 'border_type'},
 {'key': 'boundary'},
 {'key': 'brand'},
 {'key': 'bridge'},
 {'key': 'building'},
 {'key': 'building_1'},
 {'key': 'bus'},
 {'key': 'cable_submarine:category'},
 {'key': 'cable_submarine:name'},
 {'key': 'cables'},
 {'key': 'canoe'},
 {'key': 'capacity'},
 {'key': 'city'},
 {'key': 'clothes'},
 {'key': 'colour'},
 

In [30]:
sql_query("""
    SELECT *
    FROM ways_tags
    WHERE id = 183988544
""")

[{'id': 183988544,
  'key': 'name',
  'type': 'regular',
  'value': 'Casa José de Alencar (CJA)'},
 {'id': 183988544,
  'key': 'opening_hours',
  'type': 'regular',
  'value': 'Mo-Su 08:00-18:00'},
 {'id': 183988544, 'key': 'tourism', 'type': 'regular', 'value': 'attraction'}]


In [24]:
sql_query("""
    SELECT *
    FROM ways_tags
    WHERE key = 'tourism'
""")

[{'id': 24419367, 'key': 'tourism', 'type': 'regular', 'value': 'viewpoint'},
 {'id': 183988544, 'key': 'tourism', 'type': 'regular', 'value': 'attraction'},
 {'id': 220843389, 'key': 'tourism', 'type': 'regular', 'value': 'attraction'},
 {'id': 243208884, 'key': 'tourism', 'type': 'regular', 'value': 'hotel'},
 {'id': 243224678, 'key': 'tourism', 'type': 'regular', 'value': 'hotel'},
 {'id': 243227510, 'key': 'tourism', 'type': 'regular', 'value': 'attraction'},
 {'id': 246500500, 'key': 'tourism', 'type': 'regular', 'value': 'hotel'},
 {'id': 288862588, 'key': 'tourism', 'type': 'regular', 'value': 'attraction'},
 {'id': 289963904, 'key': 'tourism', 'type': 'regular', 'value': 'viewpoint'},
 {'id': 291273207, 'key': 'tourism', 'type': 'regular', 'value': 'hotel'},
 {'id': 341019399, 'key': 'tourism', 'type': 'regular', 'value': 'hotel'},
 {'id': 341019400, 'key': 'tourism', 'type': 'regular', 'value': 'hotel'},
 {'id': 354974043, 'key': 'tourism', 'type': 'regular', 'value': 'museum'

In [39]:
sql_query("""
    SELECT *
    FROM nodes
    LIMIT 5
""")

[{'changeset': 12737735,
  'id': 100428663,
  'lat': -3.7544864,
  'lon': -38.4874113,
  'timestamp': '2012-08-15T14:26:53Z',
  'uid': 804310,
  'user': 'Valentino Pinna',
  'version': 2},
 {'changeset': 46201793,
  'id': 100429090,
  'lat': -3.9356392,
  'lon': -38.3037261,
  'timestamp': '2017-02-18T20:48:54Z',
  'uid': 612405,
  'user': 'Narcélio de Sá',
  'version': 3},
 {'changeset': 10818383,
  'id': 100429099,
  'lat': -3.695213,
  'lon': -38.6172759,
  'timestamp': '2012-02-28T12:53:29Z',
  'uid': 145275,
  'user': 'Paper_',
  'version': 2},
 {'changeset': 23204765,
  'id': 100429507,
  'lat': -3.7173614,
  'lon': -38.5223213,
  'timestamp': '2014-06-27T02:56:38Z',
  'uid': 612405,
  'user': 'Narcélio de Sá',
  'version': 8},
 {'changeset': 569903,
  'id': 100430485,
  'lat': -3.8231491,
  'lon': -38.4017816,
  'timestamp': '2008-12-22T23:01:40Z',
  'uid': 1596,
  'user': 'Claudomiro',
  'version': 5}]


In [ ]:
sql_query("""
    SELECT COUNT(*)
    FROM ways_tags

## Idea for Exploration: Identify Neighborhood Saturation Level
Identify how many distinct streets are in each neighborhood, then how many nodes and ways have been tagged in each neighborhood per street, and take that ratio as the saturation level for each neighborhood. This can be a flag for potential users to which neighborhoods need the most work, so they can focus their efforts here.

In [13]:
sql_query("SELECT * FROM nodes WHERE id = 479234596")

[{'changeset': 13933916,
  'id': 479234596,
  'lat': -3.7229979,
  'lon': -38.5507686,
  'timestamp': '2012-11-19T16:29:38Z',
  'uid': 99364,
  'user': 'Arthur Rodrigues',
  'version': 2}]


In [44]:
sql_query("""
    SELECT GROUP_CONCAT(DISTINCT(id)) AS ids, COUNT(*) AS count, value AS neighborhood
    FROM (SELECT * FROM nodes_tags WHERE key = 'subur' UNION ALL SELECT * FROM ways_tags WHERE key = 'subur')
    GROUP BY neighborhood
    ORDER BY count DESC
""")

[{'count': 719,
  'ids': '358701550,373769832,373769871,373769909,373769935,373769936,373769943,373790989,379687401,379687402,380017813,380017815,380017816,380017817,380017821,380017822,380017825,380017826,380017827,380017828,380017829,380017830,380017832,380017833,380017834,380017836,380017838,380017839,380017840,380017841,380017843,380017844,380017845,380017846,380017847,380017848,385746444,386889822,386889823,386889824,386889825,386889826,386889827,386889828,386889829,386889831,386889832,386889833,386889834,386889835,386889836,386889837,386889838,386889839,386889840,386889841,386889842,386889843,388091188,388168411,388168412,388168414,388168415,388168416,388168417,388168418,388168419,388168420,388168421,388168422,388168423,388168424,388168425,388168426,388168427,388168428,388168429,388168431,388168432,388168433,388168434,388168435,388168436,388168437,388168438,393078959,393078960,393078961,393078962,393078963,393078964,393078965,393078966,393078967,393078968,393078969,393078970,3930

In [89]:
sql_query("""
    SELECT
        COUNT(DISTINCT(table1.value)) AS streets_count,
        table2.value AS neighborhood,
        COUNT(*) AS tags_count,
        ROUND( (COUNT(*) * 1.0) / (COUNT(DISTINCT(table1.value)) * 1.0), 2) AS saturation_ratio
    FROM
        (SELECT * FROM nodes_tags UNION SELECT * FROM ways_tags) table1,
        (SELECT * FROM nodes_tags UNION SELECT * FROM ways_tags) table2
    WHERE
        table1.id = table2.id
        AND table1.key = 'street'
        AND table2.key = 'subur'
    GROUP BY neighborhood
    HAVING streets_count >= 5
    ORDER BY saturation_ratio
    LIMIT 5
""")

[{'neighborhood': 'Granja Lisboa',
  'saturation_ratio': 1.0,
  'streets_count': 6,
  'tags_count': 6},
 {'neighborhood': 'Marcos Freire',
  'saturation_ratio': 1.0,
  'streets_count': 9,
  'tags_count': 9},
 {'neighborhood': 'Renascer',
  'saturation_ratio': 1.0,
  'streets_count': 11,
  'tags_count': 11},
 {'neighborhood': 'Benfica',
  'saturation_ratio': 1.18,
  'streets_count': 11,
  'tags_count': 13},
 {'neighborhood': 'Messejana',
  'saturation_ratio': 1.33,
  'streets_count': 6,
  'tags_count': 8}]


In [91]:
sql_query("SELECT key FROM nodes_tags GROUP BY key")

[{'key': 'Exercicios'},
 {'key': 'GEOCODIGO'},
 {'key': 'access'},
 {'key': 'aeroway'},
 {'key': 'alt_name'},
 {'key': 'amenity'},
 {'key': 'amenity_1'},
 {'key': 'ar'},
 {'key': 'artwork_type'},
 {'key': 'atm'},
 {'key': 'attraction'},
 {'key': 'backrest'},
 {'key': 'barrier'},
 {'key': 'be'},
 {'key': 'beacon_isolated_danger:colour'},
 {'key': 'beacon_isolated_danger:colour_pattern'},
 {'key': 'beauty'},
 {'key': 'bench'},
 {'key': 'bg'},
 {'key': 'bicycle'},
 {'key': 'bicycle:chain_tool'},
 {'key': 'bicycle:pump'},
 {'key': 'bicycle_parking'},
 {'key': 'bitcoin'},
 {'key': 'bn'},
 {'key': 'br'},
 {'key': 'braille:lg'},
 {'key': 'brand'},
 {'key': 'building'},
 {'key': 'buoy_isolated_danger:colour'},
 {'key': 'buoy_isolated_danger:colour_pattern'},
 {'key': 'buoy_isolated_danger:shape'},
 {'key': 'buoy_lateral:category'},
 {'key': 'buoy_lateral:colour'},
 {'key': 'buoy_lateral:shape'},
 {'key': 'buoy_lateral:system'},
 {'key': 'bus'},
 {'key': 'capacity'},
 {'key': 'ce'},
 {'key': 'c

In [35]:
sql_query("""
    SELECT *, (SELECT GROUP_CONCAT(DISTINCT(id)) FROM (SELECT * FROM nodes_tags WHERE key = 'subur' UNION ALL SELECT * FROM ways_tags WHERE key = 'subur')
    FROM nodes,
    (
        SELECT GROUP_CONCAT(DISTINCT(id)) AS ids, COUNT(*) AS count, value AS neighborhood
        FROM (SELECT * FROM nodes_tags WHERE key = 'subur' UNION ALL SELECT * FROM ways_tags WHERE key = 'subur')
        GROUP BY neighborhood
    ) subquery1
    WHERE nodes.id IN subquery1.ids

""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



OperationalError: no such table: sub.ids

In [23]:
# number of tags per neighborhood
sql_query("""
    SELECT COUNT(*) as count, value as neighborhood
    FROM (SELECT * FROM nodes_tags WHERE key = 'subur' UNION ALL SELECT * FROM ways_tags WHERE key = 'subur')
    GROUP BY neighborhood
    ORDER BY count DESC
""")

[{'count': 719, 'neighborhood': 'Parque Albano'},
 {'count': 451, 'neighborhood': 'Montese'},
 {'count': 218, 'neighborhood': 'Prefeito José Walter'},
 {'count': 171, 'neighborhood': 'Damas'},
 {'count': 169, 'neighborhood': 'Conjunto Ceará'},
 {'count': 149, 'neighborhood': 'Parangaba'},
 {'count': 134, 'neighborhood': 'Vila União'},
 {'count': 128, 'neighborhood': 'Mondubim'},
 {'count': 108, 'neighborhood': 'Rodolfo Teófilo'},
 {'count': 89, 'neighborhood': 'Jardim América'},
 {'count': 84, 'neighborhood': 'Parque Dois Irmãos'},
 {'count': 82, 'neighborhood': 'Maraponga'},
 {'count': 67, 'neighborhood': 'Serrinha'},
 {'count': 66, 'neighborhood': 'Parquelândia'},
 {'count': 59, 'neighborhood': 'Aldeota'},
 {'count': 56, 'neighborhood': 'Alphaville Ceará III'},
 {'count': 53, 'neighborhood': 'Planalto Ayrton Senna'},
 {'count': 52, 'neighborhood': 'Meireles'},
 {'count': 50, 'neighborhood': 'Centro'},
 {'count': 46, 'neighborhood': 'Dionísio Torres'},
 {'count': 43, 'neighborhood': '

In [51]:
sql_query("""
    SELECT COUNT(DISTINCT(value)) AS num_streets
    FROM nodes_tags
    WHERE id IN
    (SELECT id FROM nodes_tags WHERE value = 'Rodolfo Teófilo')
    AND key = 'street' AND type = 'addr'
""")

[{'num_streets': 23}]


In [68]:
sql_query("""
    SELECT sub1.neighborhood, sub1.num_tags, COUNT(DISTINCT(sub2.value)) AS num_streets
    FROM
    (
        SELECT COUNT(*) as num_tags, value as neighborhood
        FROM (SELECT * FROM nodes_tags WHERE key = 'subur' UNION ALL SELECT * FROM ways_tags WHERE key = 'subur')
        GROUP BY neighborhood
    ) sub1,
    (
        SELECT id FROM nodes_tags WHERE value = sub1.neighborhood AND key = 'street' AND type = 'addr'
    ) sub2
    ORDER BY num_tags DESC
""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



OperationalError: no such column: sub1.neighborhood